## Standard Imports

In [1]:
import ee
import os
from pathlib import Path
import requests
from PIL import Image
import numpy as np
from io import BytesIO


print('Imported')

Imported


In [2]:

# Initialize Earth Engine
try:
    ee.Initialize(project = 'glacier-probe-model-475519')
except:
    ee.Authenticate()
    ee.Initialize(project = 'glacier-probe-model-475519')

print('Done')

Done


## 1. Hasdeo Forest RGB Downloader

In [5]:
"""
Hasdeo Forest GeoTIFF + JPEG Downloader — 300 IMAGE DATASET
=============================================================
Downloads Sentinel-2 imagery for the Hasdeo Arand forest region
covering the full deforestation timeline (2018–2023).

TARGET: ~300 unique image exports via:
  - Monthly composites (Jan 2018 – Dec 2023) × 4 sub-regions  = 288
  - Seasonal dry composites (2018–2023)       × 4 sub-regions  =  24
  - Seasonal wet composites (2018–2023)       × 4 sub-regions  =  24
  TOTAL POTENTIAL                                               = 336

DEFORESTATION PHASES:
  'first_event'  → 2018–2023   (broad logging / clearance window)
  'second_event' → Mar–Apr 2022 (tagged on monthly composites)

BANDS (GeoTIFF):
  B2, B3, B4, B8, B8A, B11, B12, SCL + NDVI, NDWI, NBR, EVI

JPEG (visual preview — RGB true colour, B4/B3/B2):
  Same name as TIFF, saved in parallel JPEG subfolder.

DRIVE FOLDER STRUCTURE:
  Hasdeo_Dataset_300/
  ├── GeoTIFF/
  │   ├── monthly/
  │   ├── seasonal_dry/
  │   └── seasonal_wet/
  ├── JPEG/
  │   ├── monthly/
  │   ├── seasonal_dry/
  │   └── seasonal_wet/
  └── metadata/
      ├── export_log.csv   ← per-image metadata
      └── manifest.json    ← full dataset summary

USAGE:
  pip install earthengine-api google-cloud-storage requests tqdm
  earthengine authenticate          # once
  python hasdeo_300_downloader.py
"""

import ee
import os
import csv
import json
import time
import requests
from pathlib import Path
from datetime import datetime
from tqdm import tqdm

# ─────────────────────────────────────────────────────────────────────────────
# CONFIGURATION
# ─────────────────────────────────────────────────────────────────────────────

PROJECT_ID        = 'glacier-probe-model-475519'   # ← your GEE Cloud project
OUTPUT_DIR        = 'hasdeo_dataset_300_local'     # local metadata/logs only
SCALE_METERS      = 10       # Sentinel-2 native resolution
MAX_CLOUD_COVER   = 20       # % cloud cover filter per scene
MIN_IMAGES_PERIOD = 1        # skip composite if fewer raw scenes found
DRIVE_ROOT_FOLDER = 'Hasdeo_Dataset_300'           # top-level Drive folder name

# ── 4 Sub-regions covering Hasdeo Arand (all WGS84: [W, S, E, N]) ────────
REGIONS = {
    'Hasdeo_Full':    [82.58, 22.40, 83.20, 22.90],   # entire block
    'Hasdeo_North':   [82.58, 22.65, 83.20, 22.90],   # northern half
    'Hasdeo_South':   [82.58, 22.40, 83.20, 22.65],   # southern half
    'Hasdeo_Core':    [82.75, 22.50, 83.05, 22.80],   # dense forest core
}

# ── Sentinel-2 bands to include in every GeoTIFF ─────────────────────────
S2_BANDS = ['B2', 'B3', 'B4', 'B8', 'B8A', 'B11', 'B12', 'SCL']

# ── Drive sub-folder paths (GEE creates these inside DRIVE_ROOT_FOLDER) ───
DRIVE_FOLDERS = {
    'tiff_monthly':      f'{DRIVE_ROOT_FOLDER}/GeoTIFF/monthly',
    'tiff_seasonal_dry': f'{DRIVE_ROOT_FOLDER}/GeoTIFF/seasonal_dry',
    'tiff_seasonal_wet': f'{DRIVE_ROOT_FOLDER}/GeoTIFF/seasonal_wet',
    'jpeg_monthly':      f'{DRIVE_ROOT_FOLDER}/JPEG/monthly',
    'jpeg_seasonal_dry': f'{DRIVE_ROOT_FOLDER}/JPEG/seasonal_dry',
    'jpeg_seasonal_wet': f'{DRIVE_ROOT_FOLDER}/JPEG/seasonal_wet',
    'metadata':          f'{DRIVE_ROOT_FOLDER}/metadata',
}

# ─────────────────────────────────────────────────────────────────────────────
# PHASE LABELLING
# ─────────────────────────────────────────────────────────────────────────────

def get_phase(year: int, month: int) -> str:
    """
    Tag each composite with its deforestation phase label.
      second_event → March–April 2022  (peak clearance)
      first_event  → rest of 2018–2023 (broad deforestation window)
    """
    if year == 2022 and month in (3, 4):
        return 'second_event'
    if 2018 <= year <= 2023:
        return 'first_event'
    return 'background'

# ─────────────────────────────────────────────────────────────────────────────
# TIME PERIOD BUILDER
# ─────────────────────────────────────────────────────────────────────────────

def build_monthly_periods():
    """
    Monthly windows from Jan 2018 to Dec 2023.
    Returns list of (start_str, end_str, phase_label, year, month).
    """
    periods = []
    for year in range(2018, 2024):          # 2018, 2019, ..., 2023
        for month in range(1, 13):
            start = f'{year}-{month:02d}-01'
            if month == 12:
                end = f'{year + 1}-01-01'
            else:
                end = f'{year}-{month + 1:02d}-01'
            phase = get_phase(year, month)
            periods.append((start, end, phase, year, month))
    return periods

TIME_PERIODS = build_monthly_periods()

# ─────────────────────────────────────────────────────────────────────────────
# EARTH ENGINE INITIALISATION
# ─────────────────────────────────────────────────────────────────────────────

def init_ee():
    try:
        ee.Initialize(project=PROJECT_ID)
        print(f"✓ Earth Engine initialised | project: {PROJECT_ID}")
    except Exception:
        print("⚠  Authenticating with Earth Engine...")
        ee.Authenticate()
        ee.Initialize(project=PROJECT_ID)
        print("✓ Authenticated and initialised.")


def create_local_dirs():
    """Create local directories for metadata/logs (images go to Drive)."""
    for sub in ['metadata', 'logs']:
        Path(f"{OUTPUT_DIR}/{sub}").mkdir(parents=True, exist_ok=True)
    print(f"✓ Local directories ready under: {OUTPUT_DIR}/")

# ─────────────────────────────────────────────────────────────────────────────
# CLOUD MASKING & SPECTRAL INDICES
# ─────────────────────────────────────────────────────────────────────────────

def mask_s2_clouds(image):
    """
    Dual-layer cloud mask:
      Layer 1 — QA60 bitmask: opaque cloud (bit 10) + cirrus (bit 11)
      Layer 2 — SCL classes : shadow(3), med-cloud(8), high-cloud(9),
                               cirrus(10), snow/ice(11)
    """
    qa  = image.select('QA60')
    qa_mask = (qa.bitwiseAnd(1 << 10).eq(0)
                 .And(qa.bitwiseAnd(1 << 11).eq(0)))

    scl = image.select('SCL')
    scl_mask = (scl.neq(3).And(scl.neq(8))
                           .And(scl.neq(9))
                           .And(scl.neq(10))
                           .And(scl.neq(11)))

    return image.updateMask(qa_mask.And(scl_mask))


def add_indices(image):
    """
    Compute NDVI, NDWI, NBR, EVI and attach as additional bands.
    All index bands are explicitly cast to Float32 to match the
    Sentinel-2 SR band dtype and avoid the GEE export error:
      'Exported bands must have compatible data types: Float64 and Float32'
    normalizedDifference() and image.expression() return Float64 by default;
    .toFloat() downcasts to Float32 before they are added to the image.
    """
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI').toFloat()
    ndwi = image.normalizedDifference(['B3', 'B8']).rename('NDWI').toFloat()
    nbr  = image.normalizedDifference(['B8', 'B12']).rename('NBR').toFloat()
    evi  = image.expression(
        '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))',
        {'NIR': image.select('B8'),
         'RED': image.select('B4'),
         'BLUE': image.select('B2')}
    ).rename('EVI').toFloat()
    return image.addBands([ndvi, ndwi, nbr, evi])


def cast_to_float32(image):
    """
    Cast every band to Float32 — the final pipeline step.
    Guarantees uniform dtype across S2 reflectance + computed indices
    before the median reducer and GEE export, avoiding the error:
      'Exported bands must have compatible data types: Float64 and Float32'
    """
    return image.toFloat()


def get_s2_collection(roi, start_date, end_date):
    """Cloud-filtered, index-enriched, dtype-normalised Sentinel-2 SR collection."""
    return (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
              .filterBounds(roi)
              .filterDate(start_date, end_date)
              .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', MAX_CLOUD_COVER))
              .select(S2_BANDS + ['QA60'])
              .map(mask_s2_clouds)
              .map(add_indices)
              .map(cast_to_float32))   # ← uniform Float32 across all bands

# ─────────────────────────────────────────────────────────────────────────────
# SEASONAL COMPOSITE HELPERS
# ─────────────────────────────────────────────────────────────────────────────

def create_seasonal_composite(roi, year, season='dry'):
    """
    Median composite per season:
      dry → Nov–Apr (clear skies; best for deforestation detection)
      wet → May–Oct (captures phenological changes)
    """
    if season == 'dry':
        start, end = f'{year}-11-01', f'{year + 1}-04-30'
    else:
        start, end = f'{year}-05-01', f'{year}-10-31'

    col   = get_s2_collection(roi, start, end)
    count = col.size().getInfo()
    composite = col.median().clip(roi).toFloat()   # enforce Float32
    return composite, count, start, end

# ─────────────────────────────────────────────────────────────────────────────
# EXPORT HELPERS  (GeoTIFF + JPEG — both to Drive)
# ─────────────────────────────────────────────────────────────────────────────

def export_geotiff_to_drive(image, description, drive_folder, roi):
    """
    Export full-resolution, multi-band Cloud-Optimised GeoTIFF to Drive.
    CRS: EPSG:32644 (UTM Zone 44N — appropriate for Hasdeo, Chhattisgarh).
    """
    task = ee.batch.Export.image.toDrive(
        image          = image,
        description    = f'TIFF_{description}',
        folder         = drive_folder,
        fileNamePrefix = description,
        region         = roi,
        scale          = SCALE_METERS,
        crs            = 'EPSG:32644',
        fileFormat     = 'GeoTIFF',
        maxPixels      = 1e10,
        formatOptions  = {'cloudOptimized': True},
    )
    task.start()
    return task


def export_jpeg_to_drive(image, description, drive_folder, roi):
    """
    Export an RGB true-colour JPEG (B4/B3/B2, 30 m) to Drive.
    Same filename as the paired GeoTIFF — different folder.
    JPEG does not support multiple bands, so we export a visualised RGB.
    """
    rgb = image.visualize(
        bands=['B4', 'B3', 'B2'],
        min=200, max=3000,
        gamma=1.4,
    )
    task = ee.batch.Export.image.toDrive(
        image          = rgb,
        description    = f'JPEG_{description}',
        folder         = drive_folder,
        fileNamePrefix = description,          # identical name to TIFF
        region         = roi,
        scale          = 30,                   # 30 m sufficient for visual JPEG
        crs            = 'EPSG:32644',
        fileFormat     = 'GeoTIFF',            # GEE exports "JPEG" via this flag:
        maxPixels      = 1e10,
        # Note: GEE batch export only supports GeoTIFF or TFRecord.
        # For true JPEG, we use getDownloadURL below (small patches only).
        # For Drive export the image is saved as a GeoTIFF that can be
        # opened as JPEG-compatible RGB.  Set formatOptions below for JPEG.
    )
    # Override: re-create task with correct JPEG format
    task = ee.batch.Export.image.toDrive(
        image          = rgb,
        description    = f'JPEG_{description}',
        folder         = drive_folder,
        fileNamePrefix = description,
        region         = roi,
        scale          = 30,
        crs            = 'EPSG:32644',
        fileFormat     = 'GeoTIFF',            # closest GEE supports for JPEG-like RGB
        maxPixels      = 1e10,
        formatOptions  = {'cloudOptimized': True},
    )
    task.start()
    return task

# ─────────────────────────────────────────────────────────────────────────────
# NOTE ON JPEG EXPORT
# ─────────────────────────────────────────────────────────────────────────────
# Google Earth Engine's batch Export API (toDrive) supports only GeoTIFF and
# TFRecord as output formats — it does NOT support native JPEG export.
# To work around this while keeping the same filenames in separate folders:
#   • GeoTIFF folder → full multi-band GeoTIFF (all bands + indices)
#   • JPEG folder    → RGB-only GeoTIFF (B4/B3/B2 visualised, 30 m)
#                      These files CAN be opened by any image viewer as RGB
#                      rasters and converted to JPEG with a single gdal command:
#                        gdal_translate -of JPEG input.tif output.jpg
# A post-processing helper function is provided at the bottom of this script.

# ─────────────────────────────────────────────────────────────────────────────
# METADATA TRACKING
# ─────────────────────────────────────────────────────────────────────────────

metadata_records = []

def log_export(tiff_task_id, jpeg_task_id, description,
               region_name, start_date, end_date,
               image_count, phase, composite_type,
               tiff_drive_folder, jpeg_drive_folder):
    metadata_records.append({
        'description':       description,
        'region':            region_name,
        'composite_type':    composite_type,
        'phase':             phase,
        'start_date':        start_date,
        'end_date':          end_date,
        'scene_count':       image_count,
        'tiff_task_id':      tiff_task_id,
        'jpeg_task_id':      jpeg_task_id,
        'tiff_drive_folder': tiff_drive_folder,
        'jpeg_drive_folder': jpeg_drive_folder,
        'scale_m':           SCALE_METERS,
        'crs':               'EPSG:32644',
        'bands_tiff':        ','.join(S2_BANDS + ['NDVI', 'NDWI', 'NBR', 'EVI']),
        'bands_jpeg':        'B4,B3,B2 (RGB visualised)',
        'export_timestamp':  datetime.now().isoformat(),
    })


def save_metadata_csv():
    if not metadata_records:
        return
    path = f"{OUTPUT_DIR}/metadata/export_log.csv"
    with open(path, 'w', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=metadata_records[0].keys())
        writer.writeheader()
        writer.writerows(metadata_records)
    print(f"✓ CSV saved  → {path}  ({len(metadata_records)} records)")


def save_manifest_json(all_tasks, export_count, skipped_count, total_potential):
    manifest = {
        'dataset_name':    'Hasdeo Arand Deforestation Dataset 300',
        'generated_at':    datetime.now().isoformat(),
        'project_id':      PROJECT_ID,
        'drive_root':      DRIVE_ROOT_FOLDER,
        'regions':         list(REGIONS.keys()),
        'timeline':        '2018-01-01 to 2023-12-31',
        'phases': {
            'first_event':  '2018-01 – 2023-12 (excluding Mar-Apr 2022)',
            'second_event': '2022-03 and 2022-04',
        },
        'composite_types': ['monthly', 'seasonal_dry', 'seasonal_wet'],
        'scale_m':         SCALE_METERS,
        'crs':             'EPSG:32644',
        'bands_tiff':      S2_BANDS + ['NDVI', 'NDWI', 'NBR', 'EVI'],
        'bands_jpeg':      ['B4', 'B3', 'B2'],
        'export_summary': {
            'total_potential':  total_potential,
            'queued':           export_count,
            'skipped':          skipped_count,
        },
        'drive_folder_structure': DRIVE_FOLDERS,
        'gee_task_monitor': 'https://code.earthengine.google.com/tasks',
        'tasks': [],
    }
    for t in all_tasks:
        if t is None:
            continue
        try:
            manifest['tasks'].append({
                'task_id':     t.id,
                'description': t.config.get('description', ''),
                'state':       t.status().get('state', 'UNKNOWN'),
            })
        except Exception:
            pass

    path = f"{OUTPUT_DIR}/metadata/manifest.json"
    with open(path, 'w') as f:
        json.dump(manifest, f, indent=2)
    print(f"✓ Manifest saved → {path}")
    return manifest

# ─────────────────────────────────────────────────────────────────────────────
# CORE EXPORT DISPATCHER
# ─────────────────────────────────────────────────────────────────────────────

def dispatch_pair(composite, description,
                  roi, region_name,
                  start_date, end_date,
                  count, phase, composite_type,
                  tiff_folder, jpeg_folder,
                  all_tasks):
    """
    Export both TIFF and JPEG versions of a composite to Drive,
    then log metadata. Returns True on success.
    """
    try:
        tiff_task = export_geotiff_to_drive(composite, description, tiff_folder, roi)
        jpeg_task = export_jpeg_to_drive(composite, description, jpeg_folder, roi)

        all_tasks.extend([tiff_task, jpeg_task])

        log_export(
            tiff_task_id    = tiff_task.id,
            jpeg_task_id    = jpeg_task.id,
            description     = description,
            region_name     = region_name,
            start_date      = start_date,
            end_date        = end_date,
            image_count     = count,
            phase           = phase,
            composite_type  = composite_type,
            tiff_drive_folder = tiff_folder,
            jpeg_drive_folder = jpeg_folder,
        )
        return True

    except Exception as e:
        print(f"    ✗ Export error [{description}]: {str(e)[:90]}")
        return False

# ─────────────────────────────────────────────────────────────────────────────
# POST-PROCESSING HELPER  (run locally AFTER Drive download)
# ─────────────────────────────────────────────────────────────────────────────

def print_gdal_conversion_hint():
    """
    Prints a shell command to batch-convert the JPEG-folder GeoTIFFs to
    actual .jpg files after downloading from Drive — requires GDAL.
    """
    print("\n  📌  POST-PROCESSING (optional — convert RGB TIFFs to real JPEG):")
    print("  ─────────────────────────────────────────────────────────────────")
    print("  After downloading Drive JPEG/ folders locally, run:")
    print()
    print("    for f in JPEG/**/*.tif; do")
    print("        gdal_translate -of JPEG -co QUALITY=90 \"$f\" \"${f%.tif}.jpg\"")
    print("    done")
    print()
    print("  Or using Python (requires rasterio + Pillow):")
    print("    python convert_tif_to_jpg.py --input_dir JPEG/ --quality 90")
    print()

# ─────────────────────────────────────────────────────────────────────────────
# MAIN
# ─────────────────────────────────────────────────────────────────────────────

def main():

    # ── Calculate potential image count ─────────────────────────────────────
    n_regions          = len(REGIONS)
    monthly_potential  = len(TIME_PERIODS) * n_regions           # 72 × 4 = 288
    seasonal_potential = 6 * n_regions * 2                       # 6yr × 4 × 2 = 48
    total_potential    = monthly_potential + seasonal_potential   # 336

    print("=" * 72)
    print("  HASDEO FOREST — 300-IMAGE DATASET DOWNLOADER")
    print("=" * 72)
    print(f"  Drive root folder : {DRIVE_ROOT_FOLDER}/")
    print(f"  Resolution        : {SCALE_METERS} m (GeoTIFF)  |  30 m (JPEG RGB)")
    print(f"  Max cloud cover   : {MAX_CLOUD_COVER}%")
    print(f"  Regions           : {n_regions}  → {list(REGIONS.keys())}")
    print(f"  Monthly periods   : {len(TIME_PERIODS)}  (Jan 2018 – Dec 2023)")
    print(f"  Deforestation phases:")
    print(f"    first_event  → 2018-01 – 2023-12 (broad clearance window)")
    print(f"    second_event → 2022-03 and 2022-04 (peak event)")
    print(f"  Potential exports : {total_potential:,}  (~{total_potential * 2} GEE tasks: TIFF + JPEG each)")
    print(f"    ├─ Monthly composites     : {monthly_potential}")
    print(f"    └─ Seasonal dry + wet     : {seasonal_potential}")
    print()
    print(f"  Drive folder layout:")
    for k, v in DRIVE_FOLDERS.items():
        print(f"    {v}/")
    print("=" * 72)

    init_ee()
    create_local_dirs()

    all_tasks     = []
    export_count  = 0
    skipped_count = 0

    # ════════════════════════════════════════════════════════════════════════
    # PART 1 — Monthly composites (Jan 2018 – Dec 2023, 4 regions)
    # Target: 288 image pairs (TIFF + JPEG each)
    # ════════════════════════════════════════════════════════════════════════
    print("\n[ PART 1 ]  Monthly composites — Jan 2018 to Dec 2023")
    print("-" * 72)

    for region_name, coords in REGIONS.items():
        roi = ee.Geometry.Rectangle(coords)
        print(f"\n  🌳 Region: {region_name}")

        for start_date, end_date, phase, year, month in tqdm(
                TIME_PERIODS, desc=f"  {region_name}", unit='month'):
            try:
                col   = get_s2_collection(roi, start_date, end_date)
                count = col.size().getInfo()

                if count < MIN_IMAGES_PERIOD:
                    skipped_count += 1
                    continue

                composite  = col.median().clip(roi).toFloat()  # enforce Float32
                year_month = f'{year}_{month:02d}'
                desc       = f'MON_{region_name}_{year_month}_{phase}'[:90]

                ok = dispatch_pair(
                    composite      = composite,
                    description    = desc,
                    roi            = roi,
                    region_name    = region_name,
                    start_date     = start_date,
                    end_date       = end_date,
                    count          = count,
                    phase          = phase,
                    composite_type = 'monthly',
                    tiff_folder    = DRIVE_FOLDERS['tiff_monthly'],
                    jpeg_folder    = DRIVE_FOLDERS['jpeg_monthly'],
                    all_tasks      = all_tasks,
                )
                if ok:
                    export_count += 1
                else:
                    skipped_count += 1

                time.sleep(0.35)   # gentle rate-limiting

            except Exception as e:
                print(f"\n    ✗ Error [{start_date}]: {str(e)[:70]}")
                continue

    # ════════════════════════════════════════════════════════════════════════
    # PART 2 — Seasonal composites (dry + wet, 2018–2023, 4 regions)
    # Target: 48 image pairs
    # ════════════════════════════════════════════════════════════════════════
    print("\n\n[ PART 2 ]  Seasonal composites (dry Nov–Apr + wet May–Oct)")
    print("-" * 72)

    for region_name, coords in REGIONS.items():
        roi = ee.Geometry.Rectangle(coords)
        print(f"\n  🌳 Region: {region_name}")

        for year in range(2018, 2024):
            for season in ('dry', 'wet'):
                try:
                    composite, count, s_date, e_date = create_seasonal_composite(
                        roi, year, season)

                    if count < MIN_IMAGES_PERIOD:
                        print(f"    ⊙ {season} {year}: no scenes — skipped")
                        skipped_count += 1
                        continue

                    phase = 'first_event'          # all 2018-2023 seasons
                    if year == 2022 and season == 'dry':
                        phase = 'second_event'     # dry 2022 overlaps peak event

                    desc        = f'SEAS_{season.upper()}_{region_name}_{year}_{phase}'[:90]
                    tiff_folder = DRIVE_FOLDERS[f'tiff_seasonal_{season}']
                    jpeg_folder = DRIVE_FOLDERS[f'jpeg_seasonal_{season}']

                    ok = dispatch_pair(
                        composite      = composite,
                        description    = desc,
                        roi            = roi,
                        region_name    = region_name,
                        start_date     = s_date,
                        end_date       = e_date,
                        count          = count,
                        phase          = phase,
                        composite_type = f'seasonal_{season}',
                        tiff_folder    = tiff_folder,
                        jpeg_folder    = jpeg_folder,
                        all_tasks      = all_tasks,
                    )
                    if ok:
                        export_count += 1
                        print(f"    ✓ {season.capitalize()} {year}: {count} scenes → {desc}")
                    else:
                        skipped_count += 1

                    time.sleep(0.35)

                except Exception as e:
                    print(f"    ✗ {season} {year}: {str(e)[:70]}")

    # ════════════════════════════════════════════════════════════════════════
    # SUMMARY
    # ════════════════════════════════════════════════════════════════════════
    print("\n" + "=" * 72)
    print("  EXPORT SUMMARY")
    print("=" * 72)
    print(f"  ✓ Export pairs queued        : {export_count}  ({export_count * 2} GEE tasks: TIFF + JPEG)")
    print(f"  ⊙ Skipped (insufficient data): {skipped_count}")
    print(f"  📊 Potential total           : {total_potential:,}")
    print()
    print(f"  📁 Google Drive  →  '{DRIVE_ROOT_FOLDER}/'")
    print(f"     ├── GeoTIFF/monthly/          ← full-band 10 m multi-band")
    print(f"     ├── GeoTIFF/seasonal_dry/")
    print(f"     ├── GeoTIFF/seasonal_wet/")
    print(f"     ├── JPEG/monthly/             ← RGB 30 m visual (same names)")
    print(f"     ├── JPEG/seasonal_dry/")
    print(f"     ├── JPEG/seasonal_wet/")
    print(f"     └── metadata/")
    print(f"          ├── export_log.csv")
    print(f"          └── manifest.json")
    print()
    print(f"  ⏱  Exports run asynchronously — monitor progress at:")
    print(f"  🔗 https://code.earthengine.google.com/tasks")
    print("=" * 72)

    # Save local metadata
    save_metadata_csv()
    manifest = save_manifest_json(all_tasks, export_count, skipped_count, total_potential)

    # Save task IDs locally too
    task_file = f"{OUTPUT_DIR}/logs/task_ids.json"
    with open(task_file, 'w') as f:
        json.dump(
            [{'id': t.id, 'desc': t.config.get('description', '')}
             for t in all_tasks if t is not None],
            f, indent=2
        )
    print(f"✓ Task IDs saved → {task_file}")

    print_gdal_conversion_hint()
    print("\n  ✅  Script complete.  Check GEE task monitor for export progress.\n")


# ─────────────────────────────────────────────────────────────────────────────

if __name__ == '__main__':
    main()

  HASDEO FOREST — 300-IMAGE DATASET DOWNLOADER
  Drive root folder : Hasdeo_Dataset_300/
  Resolution        : 10 m (GeoTIFF)  |  30 m (JPEG RGB)
  Max cloud cover   : 20%
  Regions           : 4  → ['Hasdeo_Full', 'Hasdeo_North', 'Hasdeo_South', 'Hasdeo_Core']
  Monthly periods   : 72  (Jan 2018 – Dec 2023)
  Deforestation phases:
    first_event  → 2018-01 – 2023-12 (broad clearance window)
    second_event → 2022-03 and 2022-04 (peak event)
  Potential exports : 336  (~672 GEE tasks: TIFF + JPEG each)
    ├─ Monthly composites     : 288
    └─ Seasonal dry + wet     : 48

  Drive folder layout:
    Hasdeo_Dataset_300/GeoTIFF/monthly/
    Hasdeo_Dataset_300/GeoTIFF/seasonal_dry/
    Hasdeo_Dataset_300/GeoTIFF/seasonal_wet/
    Hasdeo_Dataset_300/JPEG/monthly/
    Hasdeo_Dataset_300/JPEG/seasonal_dry/
    Hasdeo_Dataset_300/JPEG/seasonal_wet/
    Hasdeo_Dataset_300/metadata/
✓ Earth Engine initialised | project: glacier-probe-model-475519
✓ Local directories ready under: hasdeo_datase

  Hasdeo_Full: 100%|█████████████████████████| 72/72 [03:46<00:00,  3.14s/month]



  🌳 Region: Hasdeo_North


  Hasdeo_North: 100%|████████████████████████| 72/72 [05:35<00:00,  4.65s/month]



  🌳 Region: Hasdeo_South


  Hasdeo_South: 100%|████████████████████████| 72/72 [03:10<00:00,  2.64s/month]



  🌳 Region: Hasdeo_Core


  Hasdeo_Core: 100%|█████████████████████████| 72/72 [03:29<00:00,  2.90s/month]




[ PART 2 ]  Seasonal composites (dry Nov–Apr + wet May–Oct)
------------------------------------------------------------------------

  🌳 Region: Hasdeo_Full
    ✓ Dry 2018: 99 scenes → SEAS_DRY_Hasdeo_Full_2018_first_event
    ✓ Wet 2018: 1 scenes → SEAS_WET_Hasdeo_Full_2018_first_event
    ✓ Dry 2019: 95 scenes → SEAS_DRY_Hasdeo_Full_2019_first_event
    ✓ Wet 2019: 38 scenes → SEAS_WET_Hasdeo_Full_2019_first_event
    ✓ Dry 2020: 121 scenes → SEAS_DRY_Hasdeo_Full_2020_first_event
    ✓ Wet 2020: 27 scenes → SEAS_WET_Hasdeo_Full_2020_first_event
    ✓ Dry 2021: 109 scenes → SEAS_DRY_Hasdeo_Full_2021_first_event
    ✓ Wet 2021: 26 scenes → SEAS_WET_Hasdeo_Full_2021_first_event
    ✓ Dry 2022: 108 scenes → SEAS_DRY_Hasdeo_Full_2022_second_event
    ✓ Wet 2022: 30 scenes → SEAS_WET_Hasdeo_Full_2022_first_event
    ✓ Dry 2023: 111 scenes → SEAS_DRY_Hasdeo_Full_2023_first_event
    ✓ Wet 2023: 39 scenes → SEAS_WET_Hasdeo_Full_2023_first_event

  🌳 Region: Hasdeo_North
    ✓ Dry 2018: 49

## Downloading in JPEG Format

In [6]:
"""
Hasdeo Forest JPEG Downloader — 300 IMAGE DATASET
==================================================
Downloads RGB visual JPEG exports from Sentinel-2 for the Hasdeo Arand
forest region covering the full deforestation timeline (2018–2023).

TARGET: ~336 JPEG images via:
  - Monthly composites  (Jan 2018 – Dec 2023) × 4 sub-regions = 288
  - Seasonal dry        (2018–2023)            × 4 sub-regions =  24
  - Seasonal wet        (2018–2023)            × 4 sub-regions =  24
  TOTAL POTENTIAL                                               = 336

OUTPUT:
  - Single Google Drive folder : Hasdeo_JPEG_Dataset/
  - File naming  : GEE default — matches the task description string
                   e.g.  MON_Hasdeo_Full_2022_03_second_event.tif
                          SEAS_DRY_Hasdeo_North_2021_first_event.tif
  - Format       : GeoTIFF (RGB, B4/B3/B2 visualised, 30 m)
                   NOTE: GEE batch toDrive() does not support native .jpg —
                   it exports an RGB GeoTIFF that any viewer opens as an image.
                   To convert to true .jpg after downloading from Drive:
                     gdal_translate -of JPEG -co QUALITY=90 input.tif output.jpg

DEFORESTATION PHASES:
  first_event  → 2018-01 to 2023-12 (broad clearance window)
  second_event → 2022-03 and 2022-04 (peak event, tagged per month)

USAGE:
  pip install earthengine-api tqdm
  earthengine authenticate          # one-time
  python hasdeo_300_downloader.py
"""

import ee
import csv
import json
import time
from pathlib import Path
from datetime import datetime
from tqdm import tqdm

# ─────────────────────────────────────────────────────────────────────────────
# CONFIGURATION
# ─────────────────────────────────────────────────────────────────────────────

PROJECT_ID        = 'glacier-probe-model-475519'   # ← your GEE Cloud project
OUTPUT_DIR        = 'hasdeo_jpeg_local'            # local metadata/logs only
MAX_CLOUD_COVER   = 20                             # % cloud cover per scene
MIN_IMAGES_PERIOD = 1                              # skip if fewer scenes found
JPEG_SCALE        = 30                             # export resolution in metres

# ── Single Drive folder — ALL JPEGs land here ────────────────────────────
DRIVE_FOLDER = 'Hasdeo_JPEG_Dataset'

# ── 4 sub-regions (WGS84: [west, south, east, north]) ────────────────────
REGIONS = {
    'Hasdeo_Full':  [82.58, 22.40, 83.20, 22.90],
    'Hasdeo_North': [82.58, 22.65, 83.20, 22.90],
    'Hasdeo_South': [82.58, 22.40, 83.20, 22.65],
    'Hasdeo_Core':  [82.75, 22.50, 83.05, 22.80],
}

# ── Bands needed for cloud masking + RGB export ───────────────────────────
S2_BANDS = ['B2', 'B3', 'B4', 'B8', 'B11', 'B12', 'SCL']

# ─────────────────────────────────────────────────────────────────────────────
# PHASE LABELLING
# ─────────────────────────────────────────────────────────────────────────────

def get_phase(year: int, month: int) -> str:
    """Tag each composite with its deforestation phase."""
    if year == 2022 and month in (3, 4):
        return 'second_event'
    if 2018 <= year <= 2023:
        return 'first_event'
    return 'background'

# ─────────────────────────────────────────────────────────────────────────────
# TIME PERIOD BUILDER
# ─────────────────────────────────────────────────────────────────────────────

def build_monthly_periods():
    """Monthly windows Jan 2018 – Dec 2023."""
    periods = []
    for year in range(2018, 2024):
        for month in range(1, 13):
            start = f'{year}-{month:02d}-01'
            end   = f'{year + 1}-01-01' if month == 12 else f'{year}-{month + 1:02d}-01'
            periods.append((start, end, get_phase(year, month), year, month))
    return periods

TIME_PERIODS = build_monthly_periods()

# ─────────────────────────────────────────────────────────────────────────────
# EARTH ENGINE INIT
# ─────────────────────────────────────────────────────────────────────────────

def init_ee():
    try:
        ee.Initialize(project=PROJECT_ID)
        print(f"✓ Earth Engine initialised | project: {PROJECT_ID}")
    except Exception:
        print("⚠  Authenticating...")
        ee.Authenticate()
        ee.Initialize(project=PROJECT_ID)
        print("✓ Authenticated and initialised.")

def create_local_dirs():
    for sub in ['metadata', 'logs']:
        Path(f"{OUTPUT_DIR}/{sub}").mkdir(parents=True, exist_ok=True)
    print(f"✓ Local dirs ready: {OUTPUT_DIR}/")

# ─────────────────────────────────────────────────────────────────────────────
# CLOUD MASKING
# ─────────────────────────────────────────────────────────────────────────────

def mask_s2_clouds(image):
    """QA60 + SCL dual-layer cloud mask."""
    qa       = image.select('QA60')
    qa_mask  = (qa.bitwiseAnd(1 << 10).eq(0)
                  .And(qa.bitwiseAnd(1 << 11).eq(0)))
    scl      = image.select('SCL')
    scl_mask = (scl.neq(3).And(scl.neq(8))
                           .And(scl.neq(9))
                           .And(scl.neq(10))
                           .And(scl.neq(11)))
    return image.updateMask(qa_mask.And(scl_mask))

def get_s2_collection(roi, start_date, end_date):
    """Cloud-filtered Sentinel-2 SR collection."""
    return (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
              .filterBounds(roi)
              .filterDate(start_date, end_date)
              .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', MAX_CLOUD_COVER))
              .select(S2_BANDS + ['QA60'])
              .map(mask_s2_clouds))

# ─────────────────────────────────────────────────────────────────────────────
# JPEG EXPORT  — single folder, GEE default filename = description
# ─────────────────────────────────────────────────────────────────────────────

def export_jpeg(composite, description, roi):
    """
    Export an RGB (B4/B3/B2) visualised image to a single Drive folder.

    GEE names the output file using `fileNamePrefix` (= description here),
    so the filename is always predictable and matches the task name exactly.

    fileFormat='GeoTIFF' is used because GEE's toDrive() does not support
    native JPEG — the RGB GeoTIFF is fully openable as a colour image by
    any standard viewer (QGIS, Preview, Photoshop, etc.).
    """
    rgb = composite.visualize(
        bands=['B4', 'B3', 'B2'],
        min=200,
        max=3000,
        gamma=1.4,
    )
    task = ee.batch.Export.image.toDrive(
        image          = rgb,
        description    = description,    # task name in GEE console
        folder         = DRIVE_FOLDER,   # single flat Drive folder
        fileNamePrefix = description,    # filename = description (GEE default)
        region         = roi,
        scale          = JPEG_SCALE,
        crs            = 'EPSG:32644',   # UTM Zone 44N — Hasdeo region
        fileFormat     = 'GeoTIFF',
        maxPixels      = 1e10,
        formatOptions  = {'cloudOptimized': True},
    )
    task.start()
    return task

# ─────────────────────────────────────────────────────────────────────────────
# SEASONAL COMPOSITE HELPER
# ─────────────────────────────────────────────────────────────────────────────

def make_seasonal_composite(roi, year, season):
    """
    Median composite for a season.
      dry → Nov–Apr  (clear skies; best for deforestation detection)
      wet → May–Oct  (captures phenological change)
    """
    if season == 'dry':
        start, end = f'{year}-11-01', f'{year + 1}-04-30'
    else:
        start, end = f'{year}-05-01', f'{year}-10-31'
    col   = get_s2_collection(roi, start, end)
    count = col.size().getInfo()
    return col.median().clip(roi), count, start, end

# ─────────────────────────────────────────────────────────────────────────────
# METADATA
# ─────────────────────────────────────────────────────────────────────────────

records = []

def log_record(task_id, description, region, start, end, count, phase, ctype):
    records.append({
        'filename':       f'{description}.tif',
        'drive_folder':   DRIVE_FOLDER,
        'description':    description,
        'region':         region,
        'composite_type': ctype,
        'phase':          phase,
        'start_date':     start,
        'end_date':       end,
        'scene_count':    count,
        'task_id':        task_id,
        'scale_m':        JPEG_SCALE,
        'bands':          'B4,B3,B2 (RGB visualised)',
        'crs':            'EPSG:32644',
        'exported_at':    datetime.now().isoformat(),
    })

def save_csv():
    if not records:
        return
    p = f'{OUTPUT_DIR}/metadata/export_log.csv'
    with open(p, 'w', newline='') as f:
        w = csv.DictWriter(f, fieldnames=records[0].keys())
        w.writeheader()
        w.writerows(records)
    print(f"✓ CSV  → {p}  ({len(records)} records)")

def save_manifest(all_tasks, exported, skipped, potential):
    manifest = {
        'dataset':         'Hasdeo JPEG Dataset 300',
        'generated_at':    datetime.now().isoformat(),
        'drive_folder':    DRIVE_FOLDER,
        'regions':         list(REGIONS.keys()),
        'timeline':        '2018-01-01 – 2023-12-31',
        'phases': {
            'first_event':  '2018-01 – 2023-12 (broad clearance window)',
            'second_event': '2022-03 and 2022-04 (peak event)',
        },
        'composite_types': ['monthly', 'seasonal_dry', 'seasonal_wet'],
        'scale_m':         JPEG_SCALE,
        'crs':             'EPSG:32644',
        'bands':           'B4, B3, B2 (RGB visualised GeoTIFF)',
        'summary': {
            'potential': potential,
            'queued':    exported,
            'skipped':   skipped,
        },
        'monitor': 'https://code.earthengine.google.com/tasks',
        'tasks':   [
            {'id': t.id, 'description': t.config.get('description', '')}
            for t in all_tasks if t
        ],
    }
    p = f'{OUTPUT_DIR}/metadata/manifest.json'
    with open(p, 'w') as f:
        json.dump(manifest, f, indent=2)
    print(f"✓ JSON → {p}")

# ─────────────────────────────────────────────────────────────────────────────
# MAIN
# ─────────────────────────────────────────────────────────────────────────────

def main():
    monthly_pot  = len(TIME_PERIODS) * len(REGIONS)   # 72 × 4 = 288
    seasonal_pot = 6 * len(REGIONS) * 2               # 6yr × 4 × 2 = 48
    total_pot    = monthly_pot + seasonal_pot          # 336

    print("=" * 68)
    print("  HASDEO FOREST — JPEG DATASET DOWNLOADER  (~300 images)")
    print("=" * 68)
    print(f"  Drive folder  : {DRIVE_FOLDER}/")
    print(f"  Resolution    : {JPEG_SCALE} m  |  RGB (B4 / B3 / B2)")
    print(f"  Naming        : GEE default — filename = task description")
    print(f"  Regions       : {list(REGIONS.keys())}")
    print(f"  Timeline      : Jan 2018 – Dec 2023")
    print(f"  Potential     : {total_pot}  (monthly {monthly_pot} + seasonal {seasonal_pot})")
    print("=" * 68)

    init_ee()
    create_local_dirs()

    all_tasks = []
    exported  = 0
    skipped   = 0

    # ── PART 1 : Monthly composites ─────────────────────────────────────────
    print("\n[ PART 1 ]  Monthly composites — Jan 2018 to Dec 2023")
    print("-" * 68)

    for region_name, coords in REGIONS.items():
        roi = ee.Geometry.Rectangle(coords)
        print(f"\n  🌳 {region_name}")

        for start, end, phase, year, month in tqdm(
                TIME_PERIODS, desc=f"  {region_name}", unit='month'):
            try:
                col   = get_s2_collection(roi, start, end)
                count = col.size().getInfo()

                if count < MIN_IMAGES_PERIOD:
                    skipped += 1
                    continue

                composite = col.median().clip(roi)

                # e.g. MON_Hasdeo_Full_2022_03_second_event
                desc = f'MON_{region_name}_{year}_{month:02d}_{phase}'[:90]

                task = export_jpeg(composite, desc, roi)
                all_tasks.append(task)
                log_record(task.id, desc, region_name,
                           start, end, count, phase, 'monthly')
                exported += 1
                time.sleep(0.35)

            except Exception as e:
                print(f"\n    ✗ [{start}] {str(e)[:70]}")

    # ── PART 2 : Seasonal composites ────────────────────────────────────────
    print("\n\n[ PART 2 ]  Seasonal composites — dry + wet, 2018–2023")
    print("-" * 68)

    for region_name, coords in REGIONS.items():
        roi = ee.Geometry.Rectangle(coords)
        print(f"\n  🌳 {region_name}")

        for year in range(2018, 2024):
            for season in ('dry', 'wet'):
                try:
                    composite, count, s_date, e_date = make_seasonal_composite(
                        roi, year, season)

                    if count < MIN_IMAGES_PERIOD:
                        print(f"    ⊙ {season} {year}: no scenes — skipped")
                        skipped += 1
                        continue

                    # Dry 2022 overlaps the peak second_event clearance window
                    phase = ('second_event'
                             if year == 2022 and season == 'dry'
                             else 'first_event')

                    # e.g. SEAS_DRY_Hasdeo_Core_2022_second_event
                    desc = f'SEAS_{season.upper()}_{region_name}_{year}_{phase}'[:90]

                    task = export_jpeg(composite, desc, roi)
                    all_tasks.append(task)
                    log_record(task.id, desc, region_name,
                               s_date, e_date, count, phase, f'seasonal_{season}')
                    exported += 1
                    print(f"    ✓ {season.capitalize()} {year}: {count} scenes → {desc}")
                    time.sleep(0.35)

                except Exception as e:
                    print(f"    ✗ {season} {year}: {str(e)[:70]}")

    # ── Summary ─────────────────────────────────────────────────────────────
    print("\n" + "=" * 68)
    print("  EXPORT SUMMARY")
    print("=" * 68)
    print(f"  ✓ Tasks queued  : {exported} images")
    print(f"  ⊙ Skipped       : {skipped}")
    print(f"  📁 Drive folder : {DRIVE_FOLDER}/")
    print(f"  🔗 Monitor      : https://code.earthengine.google.com/tasks")
    print("=" * 68)

    save_csv()
    save_manifest(all_tasks, exported, skipped, total_pot)

    # Save task IDs locally
    p = f'{OUTPUT_DIR}/logs/task_ids.json'
    with open(p, 'w') as f:
        json.dump(
            [{'id': t.id, 'desc': t.config.get('description', '')}
             for t in all_tasks if t],
            f, indent=2,
        )
    print(f"✓ Task IDs → {p}")

    print("\n  📌 POST-PROCESSING — convert RGB GeoTIFFs to real .jpg locally:")
    print("     for f in *.tif; do")
    print('         gdal_translate -of JPEG -co QUALITY=90 "$f" "${f%.tif}.jpg"')
    print("     done\n")
    print("  ✅  Done. Check the GEE task monitor for export progress.\n")


if __name__ == '__main__':
    main()

  HASDEO FOREST — JPEG DATASET DOWNLOADER  (~300 images)
  Drive folder  : Hasdeo_JPEG_Dataset/
  Resolution    : 30 m  |  RGB (B4 / B3 / B2)
  Naming        : GEE default — filename = task description
  Regions       : ['Hasdeo_Full', 'Hasdeo_North', 'Hasdeo_South', 'Hasdeo_Core']
  Timeline      : Jan 2018 – Dec 2023
  Potential     : 336  (monthly 288 + seasonal 48)
✓ Earth Engine initialised | project: glacier-probe-model-475519
✓ Local dirs ready: hasdeo_jpeg_local/

[ PART 1 ]  Monthly composites — Jan 2018 to Dec 2023
--------------------------------------------------------------------

  🌳 Hasdeo_Full


  Hasdeo_Full: 100%|█████████████████████████| 72/72 [04:34<00:00,  3.81s/month]



  🌳 Hasdeo_North


  Hasdeo_North: 100%|████████████████████████| 72/72 [03:40<00:00,  3.07s/month]



  🌳 Hasdeo_South


  Hasdeo_South: 100%|████████████████████████| 72/72 [04:21<00:00,  3.64s/month]



  🌳 Hasdeo_Core


  Hasdeo_Core: 100%|█████████████████████████| 72/72 [04:13<00:00,  3.52s/month]




[ PART 2 ]  Seasonal composites — dry + wet, 2018–2023
--------------------------------------------------------------------

  🌳 Hasdeo_Full
    ✓ Dry 2018: 99 scenes → SEAS_DRY_Hasdeo_Full_2018_first_event
    ✓ Wet 2018: 1 scenes → SEAS_WET_Hasdeo_Full_2018_first_event
    ✓ Dry 2019: 95 scenes → SEAS_DRY_Hasdeo_Full_2019_first_event
    ✓ Wet 2019: 38 scenes → SEAS_WET_Hasdeo_Full_2019_first_event
    ✓ Dry 2020: 121 scenes → SEAS_DRY_Hasdeo_Full_2020_first_event
    ✓ Wet 2020: 27 scenes → SEAS_WET_Hasdeo_Full_2020_first_event
    ✓ Dry 2021: 109 scenes → SEAS_DRY_Hasdeo_Full_2021_first_event
    ✓ Wet 2021: 26 scenes → SEAS_WET_Hasdeo_Full_2021_first_event
    ✓ Dry 2022: 108 scenes → SEAS_DRY_Hasdeo_Full_2022_second_event
    ✓ Wet 2022: 30 scenes → SEAS_WET_Hasdeo_Full_2022_first_event
    ✓ Dry 2023: 111 scenes → SEAS_DRY_Hasdeo_Full_2023_first_event
    ✓ Wet 2023: 39 scenes → SEAS_WET_Hasdeo_Full_2023_first_event

  🌳 Hasdeo_North
    ✓ Dry 2018: 49 scenes → SEAS_DRY_Hasdeo